# Requirements


In [1]:
### PREREQUISTIES
### (many used only in one notebook...)

import os
import pandas as pd
import numpy as np
import pickle
import math
import random
import sys # to use your own functions
import io
import csv
from urllib.request import urlopen

import matplotlib.pyplot as plt

from scipy import stats

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.oauth2 import service_account # based on google-auth library

import sddk

In [2]:
conf = sddk.configure("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [3]:
### authorize google sheets
# to access our gsheet with data overview & metadata, you need a Google Service Account json file
# (see google-auth python package documentation to start with)
# I have mine located in my personal space on sciencedata.dk, so I read it from there:

# (1) read the file and parse its content
try:
    file_data = conf[0].get("https://sciencedata.dk/files/ServiceAccountsKey.json").json()
except:
    print("cannot find file ServiceAccountsKey.json")
# (2) transform the content into crendentials object
credentials = service_account.Credentials.from_service_account_info(file_data)
# (3) specify your usage of the credentials
scoped_credentials = credentials.with_scopes(['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive'])
# (4) use the constrained credentials for authentication of gspread package
gc = gspread.Client(auth=scoped_credentials)
# (5) establish connection with spreadsheets specified by their url
sh = gc.open_by_url("https://docs.google.com/spreadsheets/d/1boQal2mG8j_FK7XpxYM7iwIwK7Q_8ErTuor2_jC333o/edit?usp=sharing")

In [4]:
sh.worksheets()

[<Worksheet 'Sheet1' id:0>,
 <Worksheet 'seshat_axial_1000BC-1CE' id:1499875099>,
 <Worksheet 'latium_data' id:2003450966>]

# Variables

In [3]:
mg_vars = pd.read_csv("https://raw.githubusercontent.com/pesavage/moralizing-gods/master/variables.csv")
mg_vars.head(5)

,Variable,Subsection,ShortName,Category,VarNumber,Section
0,Polity Population,Social Scale,PolPop,scale,1,Social Complexity variables
1,Polity territory,Social Scale,PolTerr,scale,2,Social Complexity variables
2,Population of the largest settlement,Social Scale,CapPop,scale,3,Social Complexity variables
3,Administrative levels,Hierarchical Complexity,AdmLev,levels,4,Social Complexity variables
4,Military levels,Hierarchical Complexity,MilLev,levels,5,Social Complexity variables


In [4]:
mg_vars["Variable"].tolist()[:10]

['Polity Population',
 'Polity territory',
 'Population of the largest settlement',
 'Administrative levels',
 'Military levels',
 'Religious levels',
 'Settlement hierarchy',
 'Professional military officers',
 'Professional soldiers',
 'Professional priesthood']

In [15]:
mg_vars_dict = dict(zip(mg_vars["ShortName"], mg_vars["Variable"]))
mg_vars_dict

{'AdmLev': 'Administrative levels',
 'Afrtillery': 'Gunpowder siege artillery',
 'AgriFrontW': 'Wars on agricultural frontier',
 'Annexation': 'Annexation',
 'ArmySize': 'Army size',
 'Article': 'Articles',
 'Atlatl': 'Atlatl',
 'AudFreqDR': 'Frequency for the audience',
 'AudFreqER': 'Frequency for the audience',
 'AudFreqFR': 'Frequency for the audience',
 'AudFreqLR': 'Frequency for the audience',
 'AudFreqWR': 'Frequency for the audience',
 'BattleAxe': 'Battle Axes',
 'Breastplate': 'Breastplates',
 'Bridge': 'Bridges',
 'Bronze': 'Bronze',
 'Calendar': 'Calendar',
 'Camel': 'Camels',
 'Canal': 'Canals',
 'CapPop': 'Population of the largest settlement',
 'Chainmail': 'Chainmail',
 'CivBarbW': 'Wars on Civilization/Barbarism frontier',
 'CompBow': 'Composite bow',
 'ComplxFort': 'Complex fortifications',
 'Copper': 'Copper',
 'Courier': 'Couriers',
 'Court': 'Courts',
 'Crossbow': 'Crossbow',
 'CustomMech': 'Customary mechanism',
 'Dagger': 'Daggers',
 'DefPosition': 'Settlements 

In [16]:
mg_vars_dict_inverse = dict(zip(mg_vars["Variable"], mg_vars["ShortName"]))
mg_vars_dict_inverse

{'Administrative levels': 'AdmLev',
 'Annexation': 'Annexation',
 'Army size': 'ArmySize',
 'Articles': 'Article',
 'Atlatl': 'Atlatl',
 'Battle Axes': 'BattleAxe',
 'Breastplates': 'Breastplate',
 'Bridges': 'Bridge',
 'Bronze': 'Bronze',
 'Calendar': 'Calendar',
 'Camels': 'Camel',
 'Canals': 'Canal',
 'Chainmail': 'Chainmail',
 'Complex fortifications': 'ComplxFort',
 'Composite bow': 'CompBow',
 'Constraint on executive by government': 'ExecConsGov',
 'Constraint on executive by non-government': 'ExecConsNonGov',
 'Constraint on executive by religious agents': 'ExecConsRelig',
 'Constraint on executive by religious institutions': 'ExecConsRelInst',
 'Copper': 'Copper',
 'Corporal punishment by the state': 'SlaveCorpPunState',
 'Couriers': 'Courier',
 'Courts': 'Court',
 'Critique of executive by government': 'ExecCritGov',
 'Critique of executive by non-government': 'ExecCritNonGov',
 'Critique of executive by religious agents': 'ExecCritRelig',
 'Crossbow': 'Crossbow',
 'Customary

In [26]:
rel_variables = ["SEFair", "SERecip",  "SEIngroup", "HighGods"]
for variable_short in rel_variables:
  print(mg_vars_dict[variable_short])


supernatural enforcement of fairness
supernatural enforcement of human reciprocity
supernatural enforcement of in-group loyalty
High Gods (creator gods)


# Latest version of the complete dataset

In [0]:
url = "http://seshatdatabank.info/2018csv/data/download.csv"
response = requests.get(url).text
file_object = io.StringIO(response)
seshat_complete = pd.read_csv(file_object)

In [0]:
seshat_complete.head()

,NGA,Polity,Section,Subsection,Variable,Value From,Value To,Date From,Date To,Fact Type,Value Note,Date Note,Comment
0,Ghanaian Coast,GhAshnE,Social Complexity variables,NaN,RA,Eva Brandl,NaN,NaN,NaN,simple,simple,NaN,NaN
1,Ghanaian Coast,GhAshnE,Social Complexity variables,NaN,Editor,Daniel Mullins,NaN,NaN,NaN,simple,simple,NaN,NaN
2,Ghanaian Coast,GhAshnE,Social Complexity variables,Hierarchical Complexity,Settlement hierarchy,3,NaN,NaN,NaN,simple,simple,NaN,NaN
3,Ghanaian Coast,GhAshnE,Social Complexity variables,Hierarchical Complexity,Administrative levels,4,NaN,NaN,NaN,simple,simple,NaN,NaN
4,Ghanaian Coast,GhAshnE,Social Complexity variables,Hierarchical Complexity,Religious levels,3,NaN,NaN,NaN,simple,simple,NaN,NaN


In [0]:
seshat_complete[seshat_complete["NGA"]=="Latium"]

,NGA,Polity,Section,Subsection,Variable,Value From,Value To,Date From,Date To,Fact Type,Value Note,Date Note,Comment
15030,Latium,ItLatCA,Social Complexity variables,NaN,RA,Enrico Cioni,NaN,NaN,NaN,complex,list,NaN,NaN
15031,Latium,ItLatCA,Social Complexity variables,NaN,RA,Edward A L Turner,NaN,NaN,NaN,complex,list,NaN,NaN
15032,Latium,ItLatCA,Social Complexity variables,Hierarchical Complexity,Settlement hierarchy,1,NaN,NaN,NaN,simple,simple,NaN,NaN
15033,Latium,ItLatCA,Social Complexity variables,Hierarchical Complexity,Administrative levels,1,NaN,NaN,NaN,simple,simple,NaN,NaN
15034,Latium,ItLatCA,Social Complexity variables,Hierarchical Complexity,Religious levels,1,NaN,NaN,NaN,simple,simple,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16590,Latium,ItPapM2,Social Complexity variables,Other,Store of wealth,present,NaN,NaN,NaN,simple,simple,NaN,NaN
16591,Latium,ItPapM2,Social Complexity variables,Other,Couriers,present,NaN,NaN,NaN,simple,simple,NaN,NaN
16592,Latium,ItPapM2,Social Complexity variables,Other,Postal stations,absent,NaN,NaN,NaN,simple,simple,NaN,NaN
16593,Latium,ItPapM2,Social Complexity variables,Other,General postal service,absent,NaN,NaN,NaN,simple,simple,NaN,NaN


In [0]:
variables = seshat_complete["Variable"].unique()
variables

array(['RA', 'Editor', 'Settlement hierarchy', 'Administrative levels',
       'Religious levels', 'Military levels', 'Occupational complexity',
       'Professional military officers', 'Professional soldiers',
       'Professional priesthood', 'Full-time bureaucrats',
       'Source of support', 'Examination system', 'Merit promotion',
       'Specialized government buildings', 'Formal legal code', 'Judges',
       'Courts', 'Professional Lawyers', 'Communal buildings',
       'Special purpose house', 'Utilitarian public buildings',
       'irrigation systems', 'drinking water supply systems', 'markets',
       'food storage sites', 'Symbolic buildings',
       'Entertainment buildings', 'Knowledge/information buildings',
       'Roads', 'Bridges', 'Canals', 'Ports', 'Special purpose sites',
       'Ceremonial site', 'Burial site', 'Trading emporia', 'Enclosures',
       'Mines or quarries', 'Other site', 'Length', 'Area', 'Volume',
       'Weight', 'Time', 'Geometrical', 'Other', 'Mn

In [0]:
NGAs = seshat_complete["NGA"].unique()
NGAs

array(['Ghanaian Coast', 'Niger Inland Delta', 'Upper Egypt', 'Iceland',
       'Paris Basin', 'Yemeni Coastal Plain', 'Konya Plain', 'Susiana',
       'Garo Hills', 'Orkhon Valley', 'Sogdiana', 'Kapuasi Basin',
       'Latium', 'Cambodian Basin', 'Kachi Plain', 'Southern China Hills',
       'Middle Yellow River Valley', 'Finger Lakes', 'Central Java',
       'Kansai', 'Lena River Valley', 'Deccan', 'Lowland Andes', 'Cuzco',
       'North Colombia', 'Oro PNG', 'Cahokia', 'Big Island Hawaii',
       'Valley of Oaxaca', 'Chuuk Islands'], dtype=object)

In [0]:
len(NGAs)

30

In [0]:
for region in NGAs:
  for date in seshat_complete[seshat_complete["NGA"]==region]["Date From"].tolist():
    print(region, date) ###seshat_complete[seshat_complete["Variable"]==variable][["NGA", "Date From", "Variable", "Value From"]].values)

Streaming output truncated to the last 5000 lines.
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai 1185CE
Kansai 1226CE
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai 1300CE
Kansai 1300CE
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Kansai nan
Ka